<a href="https://colab.research.google.com/github/anindita/dances/blob/main/analyse/segmentation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentation 2

This has been modified by code written by [tugstugi](https://github.com/tugstugi/dl-colab-notebooks)

In [ ]:
YOUTUBE_ID = 'sb82tVOq2dY'

!pip install -q youtube-dl
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
!ffmpeg -y -loglevel info -i youtube.mp4 -ss 00:00:58 -t 2 video.mp4

In [ ]:
!mkdir images
video_size = (320,240)

In [ ]:

import cv2
import os
import numpy as np
  

cam = cv2.VideoCapture("video.mp4")
currentframe = 0
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
  
while(True):
    ret,frame = cam.read()
  
    if ret:
        name = '/content/images/' + str(currentframe) + '.jpg'
        print ('Creating...' + name)
  
        frame = cv2.resize(frame, (1280,720))
        frame = frame[0:720, 160:1120]
        frame = cv2.resize(frame, video_size)
        frame = cv2.filter2D(frame, -1, kernel)
        cv2.imwrite(name, frame)

        currentframe += 1
    else:
        break
  
cam.release()
cv2.destroyAllWindows()

# Prepare seg model


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CSAILVision/semantic-segmentation-pytorch.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q $git_repo_url
  #!cd $project_name && pip install -q -r requirement.txt
  
import sys
sys.path.append(project_name)
models = f'{project_name}/mit_semseg/'
sys.path.append(models)

import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

In [ ]:
ENCODER_NAME = 'resnet101'
DECODER_NAME = 'upernet'
PRETRAINED_ENCODER_MODEL_URL = 'http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-%s-%s/encoder_epoch_50.pth' % (ENCODER_NAME, DECODER_NAME)
PRETRAINED_DECODER_MODEL_URL = 'http://sceneparsing.csail.mit.edu/model/pytorch/ade20k-%s-%s/decoder_epoch_50.pth' % (ENCODER_NAME, DECODER_NAME)

pretrained_encoder_file = basename(PRETRAINED_ENCODER_MODEL_URL)
if not exists(pretrained_encoder_file):
  !wget -q $PRETRAINED_ENCODER_MODEL_URL
pretrained_decoder_file = basename(PRETRAINED_DECODER_MODEL_URL)
if not exists(pretrained_decoder_file):
  !wget -q $PRETRAINED_DECODER_MODEL_URL

In [ ]:
from types import SimpleNamespace
import torch
from models import ModelBuilder, SegmentationModule
from dataset import TestDataset
from utils import colorEncode
from scipy.io import loadmat

# options
options = SimpleNamespace(fc_dim=2048,
                          num_class=150,
                          imgSizes = [300, 400, 500, 600],
                          imgMaxSize=1000,
                          padding_constant=8,
                          segm_downsampling_rate=8)

# create model
builder = ModelBuilder()
net_encoder = builder.build_encoder(arch=ENCODER_NAME, weights=pretrained_encoder_file,
                                    fc_dim=options.fc_dim)
net_decoder = builder.build_decoder(arch=DECODER_NAME, weights=pretrained_decoder_file,
                                    fc_dim=options.fc_dim, num_class=options.num_class, use_softmax=True)
segmentation_module = SegmentationModule(net_encoder, net_decoder, torch.nn.NLLLoss(ignore_index=-1))
segmentation_module = segmentation_module.eval()
torch.set_grad_enabled(False)

if torch.cuda.is_available():
  segmentation_module = segmentation_module.cuda()

# test on a given image
def test(test_image_name):
  dataset_test = TestDataset([{'fpath_img': test_image_name}], options, max_sample=-1)
  
  batch_data = dataset_test[0]
  segSize = (batch_data['img_ori'].shape[0], batch_data['img_ori'].shape[1])
  img_resized_list = batch_data['img_data']
  
  scores = torch.zeros(1, options.num_class, segSize[0], segSize[1])
  if torch.cuda.is_available():
    scores = scores.cuda()

  for img in img_resized_list:
    feed_dict = batch_data.copy()
    feed_dict['img_data'] = img
    del feed_dict['img_ori']
    del feed_dict['info']
    if torch.cuda.is_available():
      feed_dict = {k: o.cuda() for k, o in feed_dict.items()}

    # forward pass
    pred_tmp = segmentation_module(feed_dict, segSize=segSize)
    scores = scores + pred_tmp / len(options.imgSizes)

    _, pred = torch.max(scores, dim=1)
    return pred.squeeze(0).cpu().numpy()

# Trial image

In [ ]:
def segment(image_file):
  t = time.time()
  pred = test(image_file)
  print("executed in %.3fs" % (time.time()-t))

  pred_color = colorEncode(pred, loadmat(os.path.join(project_name, 'data/color150.mat'))['colors'])
  plt.figure(figsize=(10, 5))
  plt.imshow(pred_color)

arr = os.listdir('/content/images')
for f in arr:
  segment(f'/content/images/{f}')